This is version 2 of my Fantasy Hockey Analyzer. The purpose of this notebook is to predict the number of fantasy points every hockey player in the league will get based on previous years' performance.

This notebook primarily uses data from moneypuck.com for analysis, and it also uses data from rotowire.com to get +/- for each player.

Section 1: Parameters and Modules

These are the variables that can be adjusted. My model is an ensemble model consisting of neural nets and random forests, with data going back one, two, and three years.

In [ ]:
# Set these values to the appropriate ammonts

current_year = 2025
common_number = 0
number_of_one_year_neural_nets = common_number
number_of_two_year_neural_nets = common_number
number_of_three_year_neural_nets = common_number
number_of_one_year_random_forests = common_number
number_of_two_year_random_forests = common_number
number_of_three_year_random_forests = common_number

# Sets whether any models should be deleted and written over
create_new_models = True

# This is the breakdown of how many fantasy points a player gets for each category
points_dictionary = {
    'Goals':5, 
    'Assists':3, 
    '+/-':1.5, 
    'PIM':-0.25, 
    'PP_Goals':4, 
    'PP_Assists':2, 
    'SH_Goals':6, #won't count SHG from 5-on-3
    'SH_Assists':4, 
    'Faceoffs_Won':0.25, 
    'Faceoffs_Lost':-0.15, 
    'Hits':0.5, 
    'Blocked_Shots':0.75
    }

The following is a list of modules that I used and the reason why they were used:

-os: to allow the program to read data in the repository

-numpy: basic math operations

-pandas: all dataframe operations/data storage/data cleaning

-various sklearn: all machine learning operations/analysis

In addition to these modules, I also have a custom module that contains helper functions that help in data cleaning/accuracy evaluation. These functions are contained in the "my_module.py" file in the repository. If you are interested in taking a look at these functions, they are available at https://github.com/chrisberry888/FantasyHockeyAnalyzer in the "my_module.py" file.

In [ ]:
#Import block
import os
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.base import clone

import my_module_v2 as mx

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

Section 2: Data Gathering and Cleaning

This section compiles the Moneypuck and Rotowire data into a format that is usable by the ML models.

There is some discrepencies between stats on ESPN and on moneypuck. These shouldn't alter the fantasy points too much. For example, Sydney Crosby is short-changed 1 faceoff win but has one additional hit in moneypuck than on ESPN, so in my model he has 0.10 more points than he does on ESPN. I would like to see why this is the case in the future/get fully accurate data (perhaps from ESPN themselves), but for now I am ok with this very small error.

In [ ]:
yearly_player_data = []

for year in range(2010, current_year):
    moneypuck_data = mx.get_moneypuck_data(year)
    rotowire_data = mx.get_rotowire_data(year)
    combined_data = mx.combine_dataframes(moneypuck_data, rotowire_data)
    this_years_data = mx.calculate_fantasy_points(combined_data, points_dictionary)
    yearly_player_data.append(this_years_data)

    

In [4]:
player_id_table = mx.get_player_id_table(yearly_player_data)

In [5]:
player_id_table.head()

,playerId,name
838,8484406,Ty Mueller_VAN_2024_C
839,8484768,Zayne Parekh_CGY_2024_D
856,8485512,Tim Washe_ANA_2024_C
855,8485511,Quinn Hutson_EDM_2024_R
837,8484321,Nikolas Matinpalo_OTT_2024_D


This next cell compiles the yearly data into chunks of one, two, and three-year data to be used by the ML models.

In [6]:
ml_data_one_year = mx.get_ml_data(yearly_player_data, current_year, 1)
ml_data_two_years = mx.get_ml_data(yearly_player_data, current_year, 2)
ml_data_three_years = mx.get_ml_data(yearly_player_data, current_year, 3)

/home/chris/Git_Repositories/FantasyHockeyAnalyzer/my_module_v2.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/home/chris/Git_Repositories/FantasyHockeyAnalyzer/my_module_v2.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/home/chris/Git_Repositories/FantasyHockeyAnalyzer/my_module_v2.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, se

The data is now ready to be used to train the ML model.

In [7]:

one_year_X, one_year_y = mx.separate_fantasy_points(ml_data_one_year)
two_year_X, two_year_y = mx.separate_fantasy_points(ml_data_two_years)
three_year_X, three_year_y = mx.separate_fantasy_points(ml_data_three_years)

In [3]:
# ml_data_one_year.head()

In [ ]:
one_year_neural_net_args = (
    one_year_X,
    one_year_y,
    MLPRegressor(max_iter=1000),
    3 #number_of_one_year_neural_nets
)

one_year_random_forest_args = (
    one_year_X,
    one_year_y,
    RandomForestRegressor(),
    number_of_one_year_random_forests
)

two_year_neural_net_args = (
    two_year_X,
    two_year_y,
    MLPRegressor(max_iter=1000),
    number_of_two_year_neural_nets
)

two_year_random_forest_args = (
    two_year_X,
    two_year_y,
    RandomForestRegressor(),
    number_of_two_year_random_forests
)

three_year_neural_net_args = (
    three_year_X,
    three_year_y,
    MLPRegressor(max_iter=1000),
    number_of_three_year_neural_nets
)

three_year_random_forest_args = (
    three_year_X,
    three_year_y,
    RandomForestRegressor(),
    number_of_three_year_random_forests
)

In [ ]:

if create_new_models:
    one_year_neural_nets = mx.create_models(*one_year_neural_net_args)
    one_year_random_forests = mx.create_models(*one_year_random_forest_args)
    two_year_neural_nets = mx.create_models(*two_year_neural_net_args)
    two_year_random_forests = mx.create_models(*two_year_random_forest_args)
    three_year_neural_nets = mx.create_models(*three_year_neural_net_args)
    three_year_random_forests = mx.create_models(*three_year_random_forest_args)
    


Now we generate the table with final predictions.

In [15]:
current_one_year_X = mx.get_final_year_data(yearly_player_data, 1)
current_two_year_X = mx.get_final_year_data(yearly_player_data, 2)
current_three_year_X = mx.get_final_year_data(yearly_player_data, 3)

In [2]:

display(current_one_year_X.head())

NameError: name 'current_one_year_X' is not defined

In [23]:
one_year_neural_net_predictions = mx.get_prediction_table(one_year_neural_nets, current_one_year_X, player_id_table)


In [32]:
display(one_year_neural_net_predictions.reset_index(drop=True))

,playerId,name,prediction
0,8477492,Nathan MacKinnon_COL_2024_C,9252.990798
1,8480069,Cale Makar_COL_2024_D,8125.758520
2,8471675,Sidney Crosby_PIT_2024_C,8061.446716
3,8479318,Auston Matthews_TOR_2024_C,7889.210257
4,8480002,Nico Hischier_NJD_2024_C,7789.406693
5,8476389,Vincent Trocheck_NYR_2024_C,7739.489347
6,8478427,Sebastian Aho_CAR_2024_C,7663.522845
7,8477933,Sam Reinhart_FLA_2024_C,7561.912032
8,8478420,Mikko Rantanen_DAL_2024_R,7483.764327
9,8477946,Dylan Larkin_DET_2024_C,7453.659494


In [1]:
len(one_year_neural_nets)

NameError: name 'one_year_neural_nets' is not defined